In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import os
import PIL

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import cv2
import csv
import PIL


# Dataset_Path = '/content/drive/MyDrive/inaturalist_12K'
train_path = os.path.join(Dataset_Path , 'train')
test_path = os.path.join(Dataset_Path , 'val')
# test_path1 = test_path
import pathlib
Dataset_Path = pathlib.Path(Dataset_Path)
train_path = pathlib.Path(train_path)
test_path = pathlib.Path(test_path)

In [ ]:
batch_size = 64
IMAGE_SIZE =  

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="training",
    interpolation='bilinear'
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="validation",
    interpolation='bilinear'
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True,
    seed=1234,
    interpolation='bilinear'
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = Sequential(
  [
    experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    experimental.preprocessing.RandomRotation(0.2),
    experimental.preprocessing.RandomZoom(0.2),
  ]
)

# Pretrained Model

In [ ]:
def get_pretrain_conv_model(model_name):
    if model_name == 'InceptionV3':
        base_model = tf.keras.applications.inception_v3(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
    if model_name == 'InceptionResNetV2':
        base_model = tf.keras.applications.inception_resnet_v2(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
    if model_name == 'ResNet50':
        base_model = tf.keras.applications.resnet50(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
    if model_name == 'Xception':
        base_model = tf.keras.applications.xception(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
    base_model.trainable = False
    preprocess_input = base_model.preprocess_input
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(10)
    inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
    initial_epochs = 10
    history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=validation_ds)
    return model,history
    
    
    
    

# Fine tuning